In [1]:
import pandas as pd

import sys
sys.path.append('../utils')
import classifier_utils

sys.path.append('../../lib')
from GerVADER.vaderSentimentGER import SentimentIntensityAnalyzer

import torch
from transformers import pipeline

from germansentiment import SentimentModel

/home/pb/Dokumente/diss/code/shap_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Data

In [2]:
data_path = "../../data/fine-tuning/"

In [3]:
train_df = pd.read_csv(data_path + "moravian_deu_train.csv")
train_df

,source,text,label
0,11,Ich küßte Ihm im Geist die Hand mit ihren Näge...,positive
1,37,Ihm warst du ganz geweiht!,positive
2,19,Ein Soldaten Bruder der bey meinem Meister im ...,positive
3,34,So weit ihr eigener Aufsatz.,neutral
4,21,da er noch ein paar Jahr als Gärtner u. nun 23...,negative
...,...,...,...
1763,5,Sie werden weder Hungern noch Dürsten - Denn i...,positive
1764,34,"Ach den Freund kanst du nun sehn, an dem du ge...",positive
1765,1,Sie bereuete auch hernach die Zeit oftt mit vi...,negative
1766,33,ich lies es darauf ankommen,neutral


In [4]:
test_df = pd.read_csv(data_path + "moravian_deu_test.csv")
test_df

,source,text,label
0,23,17 waren wir in Manakory,neutral
1,33,"ich grigte auch erlaubniß,",positive
2,11,"Wie mir dabey gewesen, deß waren alle meine Gl...",positive
3,26,In der Stunde machte sich der Heiland mir beka...,positive
4,27,"1744, am vierten Januar, wurde ich nebst meine...",neutral
...,...,...,...
437,9,Die lezte Zeit kränckelte er.,negative
438,38,Ich sahe sie oft auf ihrem Angesicht liegen un...,positive
439,13,Ich bat den Heiland mein Herz mit seinem Blute...,positive
440,34,nie hört’ man dich klagen;,positive


# Analyze with Models

In [5]:
prediction_path = "../../data/predictions/"

## Off-the-shelf Lexicon-based Approaches

### SentiWS

#### Build Dict

In [6]:
sentiws_base_path = "../../models/lexical-resources/SentiWS_v2.0/SentiWS_v2.0"
sentiws_df = pd.concat([
    pd.read_csv(f"{sentiws_base_path}_{label}.txt", sep="\t", names=["lemma", "polarity", "forms"]) for label in ["Positive", "Negative"]
])
sentiws_df

,lemma,polarity,forms
0,Abmachung|NN,0.0040,Abmachungen
1,Abschluß|NN,0.0040,"Abschluss,Abschlusse,Abschlusses,Abschlüsse,Ab..."
2,Abstimmung|NN,0.0040,Abstimmungen
3,Agilität|NN,0.0040,NaN
4,Aktivität|NN,0.0040,Aktivitäten
...,...,...,...
1822,übersehen|VVINF,-0.5128,"übersehe,übersiehst,übersieht,überseht,überseh..."
1823,übertreiben|VVINF,-0.0372,"übertreibe,übertreibst,übertreibt,übertreibest..."
1824,übertreten|VVINF,-0.3290,"übertrete,übertrittst,übertritt,übertretet,übe..."
1825,übertrieben|ADJX,-0.3203,"übertriebenes,übertriebenerer,übertriebeneres,..."


In [7]:
def join_columns(columns, separator):
    return separator.join([column for column in columns if column != ""])

# split concatenated lemma into lemma/pos
sentiws_df[["lemma", "pos"]] = sentiws_df["lemma"].str.split("|", expand=True)
sentiws_df

# split forms into multiple columns
sentiws_df["forms"].fillna("", inplace=True)
sentiws_df["forms"] = sentiws_df.apply(lambda x: join_columns([x["lemma"], x["forms"]], ","), axis=1)
sentiws_df["forms"] = sentiws_df["forms"].str.split(",")
sentiws_df = sentiws_df.explode("forms")
sentiws_df.rename(columns={"forms": "form"}, inplace=True)
sentiws_df

,lemma,polarity,form,pos
0,Abmachung,0.0040,Abmachung,NN
0,Abmachung,0.0040,Abmachungen,NN
1,Abschluß,0.0040,Abschluß,NN
1,Abschluß,0.0040,Abschluss,NN
1,Abschluß,0.0040,Abschlusse,NN
...,...,...,...,...
1826,überwältigen,-0.0048,überwältigte,VVINF
1826,überwältigen,-0.0048,überwältigtest,VVINF
1826,überwältigen,-0.0048,überwältigst,VVINF
1826,überwältigen,-0.0048,überwältiget,VVINF


In [8]:
sentiws = sentiws_df.set_index("form")["polarity"].to_dict()
sentiws

{'Abmachung': 0.004,
 'Abmachungen': 0.004,
 'Abschluß': 0.004,
 'Abschluss': 0.004,
 'Abschlusse': 0.004,
 'Abschlusses': 0.004,
 'Abschlüsse': 0.004,
 'Abschlüssen': 0.004,
 'Abstimmung': 0.004,
 'Abstimmungen': 0.004,
 'Agilität': 0.004,
 'Aktivität': 0.004,
 'Aktivitäten': 0.004,
 'Aktualisierung': 0.004,
 'Aktualisierungen': 0.004,
 'Aktualität': 0.004,
 'Aktualitäten': 0.004,
 'Akzeptanz': 0.004,
 'Akzeptanzen': 0.004,
 'Andrang': 0.004,
 'Andrangs': 0.004,
 'Andranges': 0.004,
 'Andrange': 0.004,
 'Anerkennung': 0.0855,
 'Anerkennungen': 0.0855,
 'Angebot': 0.0711,
 'Angeboten': 0.0711,
 'Angebote': 0.0711,
 'Angebots': 0.0711,
 'Angebotes': 0.0711,
 'Angemessenheit': 0.004,
 'Anhebung': 0.004,
 'Anhebungen': 0.004,
 'Anheiterung': 0.004,
 'Ankurbelung': 0.004,
 'Ankurbelungen': 0.004,
 'Annehmlichkeit': 0.004,
 'Annehmlichkeiten': 0.004,
 'Annäherung': 0.004,
 'Annäherungen': 0.004,
 'Anpassung': 0.004,
 'Anpassungen': 0.004,
 'Anpassungsfähigkeit': 0.004,
 'Anpassungsfähigkeit

#### Predict Sentiment

In [9]:
sentiws_train_df = train_df[["source", "text"]].copy()
sentiws_train_df["polarity"] = sentiws_train_df["text"].apply(lambda x: classifier_utils.calc_polarity(sentiws, x))
sentiws_train_df

,source,text,polarity
0,11,Ich küßte Ihm im Geist die Hand mit ihren Näge...,0.000000
1,37,Ihm warst du ganz geweiht!,0.000000
2,19,Ein Soldaten Bruder der bey meinem Meister im ...,0.014444
3,34,So weit ihr eigener Aufsatz.,0.000000
4,21,da er noch ein paar Jahr als Gärtner u. nun 23...,0.032749
...,...,...,...
1763,5,Sie werden weder Hungern noch Dürsten - Denn i...,0.005958
1764,34,"Ach den Freund kanst du nun sehn, an dem du ge...",0.000725
1765,1,Sie bereuete auch hernach die Zeit oftt mit vi...,0.000000
1766,33,ich lies es darauf ankommen,0.000000


In [10]:
sentiws_train_df.to_csv(prediction_path + "moravian_deu_test_SentiWS.csv", index=False)

In [11]:
sentiws_test_df = test_df[["source", "text"]].copy()
sentiws_test_df["polarity"] = sentiws_test_df["text"].apply(lambda x: classifier_utils.calc_polarity(sentiws, x))
sentiws_test_df

,source,text,polarity
0,23,17 waren wir in Manakory,0.000000
1,33,"ich grigte auch erlaubniß,",0.000000
2,11,"Wie mir dabey gewesen, deß waren alle meine Gl...",-0.000017
3,26,In der Stunde machte sich der Heiland mir beka...,0.000222
4,27,"1744, am vierten Januar, wurde ich nebst meine...",0.000000
...,...,...,...
437,9,Die lezte Zeit kränckelte er.,0.000000
438,38,Ich sahe sie oft auf ihrem Angesicht liegen un...,0.000000
439,13,Ich bat den Heiland mein Herz mit seinem Blute...,0.004317
440,34,nie hört’ man dich klagen;,0.000000


In [12]:
sentiws_test_df.to_csv(prediction_path + "moravian_deu_test_SentiWS.csv", index=False)

### GerVADER

In [13]:
gervader_analyzer = SentimentIntensityAnalyzer()

In [14]:
gervader_train_df = train_df[["source", "text"]].copy()
gervader_train_df["polarity"] = gervader_train_df["text"].apply(lambda x: gervader_analyzer.polarity_scores(x)["compound"])
gervader_train_df

,source,text,polarity
0,11,Ich küßte Ihm im Geist die Hand mit ihren Näge...,0.4939
1,37,Ihm warst du ganz geweiht!,0.0000
2,19,Ein Soldaten Bruder der bey meinem Meister im ...,0.6124
3,34,So weit ihr eigener Aufsatz.,0.0000
4,21,da er noch ein paar Jahr als Gärtner u. nun 23...,0.8845
...,...,...,...
1763,5,Sie werden weder Hungern noch Dürsten - Denn i...,0.5862
1764,34,"Ach den Freund kanst du nun sehn, an dem du ge...",0.8166
1765,1,Sie bereuete auch hernach die Zeit oftt mit vi...,0.0000
1766,33,ich lies es darauf ankommen,0.0000


In [15]:
gervader_train_df.to_csv(prediction_path + "moravian_deu_test_GerVADER.csv", index=False)

In [16]:
gervader_test_df = test_df[["source", "text"]].copy()
gervader_test_df["polarity"] = gervader_test_df["text"].apply(lambda x: gervader_analyzer.polarity_scores(x)["compound"])
gervader_test_df

,source,text,polarity
0,23,17 waren wir in Manakory,0.0000
1,33,"ich grigte auch erlaubniß,",0.0000
2,11,"Wie mir dabey gewesen, deß waren alle meine Gl...",0.0258
3,26,In der Stunde machte sich der Heiland mir beka...,0.5719
4,27,"1744, am vierten Januar, wurde ich nebst meine...",-0.3400
...,...,...,...
437,9,Die lezte Zeit kränckelte er.,0.0000
438,38,Ich sahe sie oft auf ihrem Angesicht liegen un...,0.0000
439,13,Ich bat den Heiland mein Herz mit seinem Blute...,0.5719
440,34,nie hört’ man dich klagen;,-0.2500


In [17]:
gervader_test_df.to_csv(prediction_path + "moravian_deu_test_GerVADER.csv", index=False)

## Off-the-shelf Transformer Models

In [18]:
gpu = -1 
if torch.cuda.is_available():
    torch.device("cuda")
    gpu = torch.cuda.current_device()

### ger-senti-bert

In [19]:
ger_senti_bert = SentimentModel()

In [20]:
ger_senti_bert_train_df = train_df[["source", "text"]].copy()
ger_senti_bert_train_df["label"] = ger_senti_bert_train_df["text"].apply(lambda x: ger_senti_bert.predict_sentiment([x])[0])
ger_senti_bert_train_df

,source,text,label
0,11,Ich küßte Ihm im Geist die Hand mit ihren Näge...,neutral
1,37,Ihm warst du ganz geweiht!,neutral
2,19,Ein Soldaten Bruder der bey meinem Meister im ...,neutral
3,34,So weit ihr eigener Aufsatz.,neutral
4,21,da er noch ein paar Jahr als Gärtner u. nun 23...,neutral
...,...,...,...
1763,5,Sie werden weder Hungern noch Dürsten - Denn i...,neutral
1764,34,"Ach den Freund kanst du nun sehn, an dem du ge...",neutral
1765,1,Sie bereuete auch hernach die Zeit oftt mit vi...,neutral
1766,33,ich lies es darauf ankommen,negative


In [21]:
ger_senti_bert_train_df.to_csv(prediction_path + "moravian_deu_test_ger-senti-bert.csv", index=False)

In [22]:
ger_senti_bert_test_df = test_df[["source", "text"]].copy()
ger_senti_bert_test_df["label"] = ger_senti_bert_test_df["text"].apply(lambda x: ger_senti_bert.predict_sentiment([x])[0])
ger_senti_bert_test_df

,source,text,label
0,23,17 waren wir in Manakory,neutral
1,33,"ich grigte auch erlaubniß,",neutral
2,11,"Wie mir dabey gewesen, deß waren alle meine Gl...",neutral
3,26,In der Stunde machte sich der Heiland mir beka...,neutral
4,27,"1744, am vierten Januar, wurde ich nebst meine...",neutral
...,...,...,...
437,9,Die lezte Zeit kränckelte er.,neutral
438,38,Ich sahe sie oft auf ihrem Angesicht liegen un...,neutral
439,13,Ich bat den Heiland mein Herz mit seinem Blute...,neutral
440,34,nie hört’ man dich klagen;,positive


In [23]:
ger_senti_bert_test_df.to_csv(prediction_path + "moravian_deu_test_ger-senti-bert.csv", index=False)

### senti-distilbert

In [24]:
senti_distilbert = pipeline("text-classification",  model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", device=gpu)

In [25]:
senti_distilbert_train_df = train_df[["source", "text"]].copy()
senti_distilbert_train_df["label"] = senti_distilbert_train_df["text"].apply(lambda x: senti_distilbert(x)[0]["label"])
senti_distilbert_train_df

,source,text,label
0,11,Ich küßte Ihm im Geist die Hand mit ihren Näge...,positive
1,37,Ihm warst du ganz geweiht!,positive
2,19,Ein Soldaten Bruder der bey meinem Meister im ...,negative
3,34,So weit ihr eigener Aufsatz.,positive
4,21,da er noch ein paar Jahr als Gärtner u. nun 23...,negative
...,...,...,...
1763,5,Sie werden weder Hungern noch Dürsten - Denn i...,positive
1764,34,"Ach den Freund kanst du nun sehn, an dem du ge...",positive
1765,1,Sie bereuete auch hernach die Zeit oftt mit vi...,negative
1766,33,ich lies es darauf ankommen,positive


In [26]:
senti_distilbert_train_df.to_csv(prediction_path + "moravian_deu_test_senti-distilbert.csv", index=False)

In [27]:
senti_distilbert_test_df = test_df[["source", "text"]].copy()
senti_distilbert_test_df["label"] = senti_distilbert_test_df["text"].apply(lambda x: senti_distilbert(x)[0]["label"])
senti_distilbert_test_df

,source,text,label
0,23,17 waren wir in Manakory,positive
1,33,"ich grigte auch erlaubniß,",negative
2,11,"Wie mir dabey gewesen, deß waren alle meine Gl...",negative
3,26,In der Stunde machte sich der Heiland mir beka...,positive
4,27,"1744, am vierten Januar, wurde ich nebst meine...",positive
...,...,...,...
437,9,Die lezte Zeit kränckelte er.,negative
438,38,Ich sahe sie oft auf ihrem Angesicht liegen un...,negative
439,13,Ich bat den Heiland mein Herz mit seinem Blute...,negative
440,34,nie hört’ man dich klagen;,negative


In [28]:
senti_distilbert_test_df.to_csv(prediction_path + "moravian_deu_test_senti-distilbert.csv", index=False)

## Fine-tuned Models

### bert*

In [29]:
finetuned_bert = pipeline("text-classification", model="../../models/fine-tuned/bert-base-cased-moravian-sentiment", device=gpu)

In [30]:
finetuned_bert_train_df = train_df[["source", "text"]].copy()
finetuned_bert_train_df["label"] = finetuned_bert_train_df["text"].apply(lambda x: finetuned_bert(x)[0]["label"])
finetuned_bert_train_df

,source,text,label
0,11,Ich küßte Ihm im Geist die Hand mit ihren Näge...,positive
1,37,Ihm warst du ganz geweiht!,positive
2,19,Ein Soldaten Bruder der bey meinem Meister im ...,positive
3,34,So weit ihr eigener Aufsatz.,neutral
4,21,da er noch ein paar Jahr als Gärtner u. nun 23...,negative
...,...,...,...
1763,5,Sie werden weder Hungern noch Dürsten - Denn i...,negative
1764,34,"Ach den Freund kanst du nun sehn, an dem du ge...",negative
1765,1,Sie bereuete auch hernach die Zeit oftt mit vi...,negative
1766,33,ich lies es darauf ankommen,neutral


In [31]:
finetuned_bert_train_df.to_csv(prediction_path + "moravian_deu_test_bert_fine-tuned.csv", index=False)

In [32]:
finetuned_bert_test_df = test_df[["source", "text"]].copy()
finetuned_bert_test_df["label"] = finetuned_bert_test_df["text"].apply(lambda x: finetuned_bert(x)[0]["label"])
finetuned_bert_test_df

,source,text,label
0,23,17 waren wir in Manakory,neutral
1,33,"ich grigte auch erlaubniß,",positive
2,11,"Wie mir dabey gewesen, deß waren alle meine Gl...",negative
3,26,In der Stunde machte sich der Heiland mir beka...,positive
4,27,"1744, am vierten Januar, wurde ich nebst meine...",neutral
...,...,...,...
437,9,Die lezte Zeit kränckelte er.,positive
438,38,Ich sahe sie oft auf ihrem Angesicht liegen un...,positive
439,13,Ich bat den Heiland mein Herz mit seinem Blute...,positive
440,34,nie hört’ man dich klagen;,positive


In [33]:
finetuned_bert_test_df.to_csv(prediction_path + "moravian_deu_test_bert_fine-tuned.csv", index=False)

### gbert*

In [34]:
finetuned_gbert = pipeline("text-classification", model="../../models/fine-tuned/gbert-base-moravian-sentiment", device=gpu)

In [35]:
finetuned_gbert_train_df = train_df[["source", "text"]].copy()
finetuned_gbert_train_df["label"] = finetuned_gbert_train_df["text"].apply(lambda x: finetuned_gbert(x)[0]["label"])
finetuned_gbert_train_df

,source,text,label
0,11,Ich küßte Ihm im Geist die Hand mit ihren Näge...,positive
1,37,Ihm warst du ganz geweiht!,positive
2,19,Ein Soldaten Bruder der bey meinem Meister im ...,positive
3,34,So weit ihr eigener Aufsatz.,neutral
4,21,da er noch ein paar Jahr als Gärtner u. nun 23...,negative
...,...,...,...
1763,5,Sie werden weder Hungern noch Dürsten - Denn i...,positive
1764,34,"Ach den Freund kanst du nun sehn, an dem du ge...",positive
1765,1,Sie bereuete auch hernach die Zeit oftt mit vi...,negative
1766,33,ich lies es darauf ankommen,neutral


In [36]:
finetuned_gbert_train_df.to_csv(prediction_path + "moravian_deu_test_gbert_fine-tuned.csv", index=False)

In [37]:
finetuned_gbert_test_df = test_df[["source", "text"]].copy()
finetuned_gbert_test_df["label"] = finetuned_gbert_test_df["text"].apply(lambda x: finetuned_gbert(x)[0]["label"])
finetuned_gbert_test_df

,source,text,label
0,23,17 waren wir in Manakory,neutral
1,33,"ich grigte auch erlaubniß,",positive
2,11,"Wie mir dabey gewesen, deß waren alle meine Gl...",positive
3,26,In der Stunde machte sich der Heiland mir beka...,positive
4,27,"1744, am vierten Januar, wurde ich nebst meine...",neutral
...,...,...,...
437,9,Die lezte Zeit kränckelte er.,negative
438,38,Ich sahe sie oft auf ihrem Angesicht liegen un...,positive
439,13,Ich bat den Heiland mein Herz mit seinem Blute...,positive
440,34,nie hört’ man dich klagen;,positive


In [38]:
finetuned_gbert_test_df.to_csv(prediction_path + "moravian_deu_test_gbert_fine-tuned.csv", index=False)

### ger-senti-bert*

In [39]:
finetuned_ger_senti_bert = pipeline("text-classification", model="../../models/fine-tuned/german-sentiment-bert-moravian-sentiment", device=gpu)

In [40]:
finetuned_ger_senti_bert_train_df = train_df[["source", "text"]].copy()
finetuned_ger_senti_bert_train_df["label"] = finetuned_ger_senti_bert_train_df["text"].apply(lambda x: finetuned_ger_senti_bert(x)[0]["label"])
finetuned_ger_senti_bert_train_df

,source,text,label
0,11,Ich küßte Ihm im Geist die Hand mit ihren Näge...,positive
1,37,Ihm warst du ganz geweiht!,positive
2,19,Ein Soldaten Bruder der bey meinem Meister im ...,positive
3,34,So weit ihr eigener Aufsatz.,neutral
4,21,da er noch ein paar Jahr als Gärtner u. nun 23...,negative
...,...,...,...
1763,5,Sie werden weder Hungern noch Dürsten - Denn i...,positive
1764,34,"Ach den Freund kanst du nun sehn, an dem du ge...",positive
1765,1,Sie bereuete auch hernach die Zeit oftt mit vi...,negative
1766,33,ich lies es darauf ankommen,neutral


In [41]:
finetuned_ger_senti_bert_train_df.to_csv(prediction_path + "moravian_deu_test_ger-senti-bert_fine-tuned.csv", index=False)

In [42]:
finetuned_ger_senti_bert_test_df = test_df[["source", "text"]].copy()
finetuned_ger_senti_bert_test_df["label"] = finetuned_ger_senti_bert_test_df["text"].apply(lambda x: finetuned_ger_senti_bert(x)[0]["label"])
finetuned_ger_senti_bert_test_df

,source,text,label
0,23,17 waren wir in Manakory,neutral
1,33,"ich grigte auch erlaubniß,",positive
2,11,"Wie mir dabey gewesen, deß waren alle meine Gl...",positive
3,26,In der Stunde machte sich der Heiland mir beka...,positive
4,27,"1744, am vierten Januar, wurde ich nebst meine...",neutral
...,...,...,...
437,9,Die lezte Zeit kränckelte er.,negative
438,38,Ich sahe sie oft auf ihrem Angesicht liegen un...,positive
439,13,Ich bat den Heiland mein Herz mit seinem Blute...,positive
440,34,nie hört’ man dich klagen;,positive


In [43]:
finetuned_ger_senti_bert_test_df.to_csv(prediction_path + "moravian_deu_test_ger-senti-bert_fine-tuned.csv", index=False)